# Project 2 - best feature selection methods
### Hubert Jaczyński, Aleksandra Kłos, Jakub Oganowski

Our primary goal is to focus on preprocessing in order to remove as many columns as possible at the beginning. We will calculate statistics, compare training and test datasets. Therefore, let us remove some columns that do not match. Additionally, we will also calculate correlation and check for outliers so that the model is not affected by them as much.

Most of the things are outlined in this comprehensive blog: https://neptune.ai/blog/tabular-data-binary-classification-tips-and-tricks-from-5-kaggle-competitions#:~:text=%2A%20Stratified%20KFold%20cross,Time%20Series%20split%20validation

First of all, let us check some features' statistics in both train and test data. Let us then compare top 10 of them:

In [2]:
import pandas as pd
import numpy as np

x_train = pd.read_csv('/content/x_train.txt', sep=r'\s+', header=None)
y_train = pd.read_csv('/content/y_train.txt', sep=r'\s+', header=None)[0]
x_test  = pd.read_csv('/content/x_test.txt',  sep=r'\s+', header=None)

stats = []
for i in x_train.columns:
    t_mean = x_train[i].mean()
    s_mean = x_test[i].mean()
    t_var  = x_train[i].var()
    s_var  = x_test[i].var()
    stats.append({
        'feature':             i,
        'train_mean':          t_mean,
        'test_mean':           s_mean,
        'mean_diff':           s_mean - t_mean,
        'train_variance':      t_var,
        'test_variance':       s_var,
        'variance_ratio':      (s_var / t_var) if t_var>0 else np.nan,
        'corr_with_target':    x_train[i].corr(y_train)
    })

summary_df = pd.DataFrame(stats).set_index('feature')

print(summary_df.head(10))



         train_mean  test_mean  mean_diff  train_variance  test_variance  \
feature                                                                    
0         15.560411  15.507078  -0.053333       18.730704      17.260227   
1         12.650449  12.655507   0.005058       14.317654      13.704232   
2         27.750084  27.736016  -0.014067       48.258792      44.019127   
3         18.796808  18.825133   0.028325       24.323544      22.632589   
4         19.071302  18.995343  -0.075959       27.471500      25.135808   
5         11.820110  11.769083  -0.051027       13.312292      12.482966   
6         19.365360  19.355964  -0.009395       28.260874      25.857634   
7         15.602632  15.517396  -0.085235       19.669401      18.001512   
8         14.163618  14.233636   0.070018       19.810112      19.051235   
9         15.989661  16.041487   0.051826       22.978933      22.298445   

         variance_ratio  corr_with_target  
feature                                    

In this part, we will look at distribution mismatches and test using Komogorov-Smirnov test. Moreover, we will look at outliers and cap them to 1st and 99th percentiles and display mismatches:

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp

q_low  = x_train.quantile(0.01)
q_high = x_train.quantile(0.99)

stats = []
for col in x_train.columns:
    ks_stat, ks_p = ks_2samp(x_train[col], x_test[col])
    train_outliers = ((x_train[col] < q_low[col]) | (x_train[col] > q_high[col])).sum()
    test_outliers  = ((x_test[col]  < q_low[col]) | (x_test[col]  > q_high[col])).sum()
    stats.append({
        'feature':        col,
        'ks_stat':        ks_stat,
        'ks_pvalue':      ks_p,
        'train_outliers': train_outliers,
        'test_outliers':  test_outliers,
        'train_q01':      q_low[col],
        'train_q99':      q_high[col],
        'train_min':      x_train[col].min(),
        'train_max':      x_train[col].max(),
        'test_min':       x_test[col].min(),
        'test_max':       x_test[col].max(),
    })

stats_df = pd.DataFrame(stats).set_index('feature')

print("KS statistics for top 10 ks_stat features:")
print(stats_df.sort_values('ks_stat', ascending=False).head(10))

x_train_capped = x_train.clip(lower=q_low, upper=q_high, axis=1)
x_test_capped  = x_test.clip( lower=q_low, upper=q_high, axis=1)

ks_after = []
for col in x_train.columns:
    ks2, p2 = ks_2samp(x_train_capped[col], x_test_capped[col])
    ks_after.append(ks2)
stats_df['ks_after_capping'] = ks_after

print("\nKS statistics for top 10 ks_stat features: after capping:")
top10 = stats_df.sort_values('ks_stat', ascending=False).head(10)
print(top10[['ks_stat','ks_after_capping']])


KS statistics for top 10 ks_stat features:
         ks_stat  ks_pvalue  train_outliers  test_outliers  train_q01  \
feature                                                                 
428       0.0426   0.000229             100            112   2.665452   
107       0.0368   0.002291             100             97  -2.315544   
130       0.0368   0.002291             100            105  -2.249431   
135       0.0354   0.003798             100            110  -2.354010   
394       0.0340   0.006174             100            156   0.012440   
303       0.0338   0.006607             100             76   0.008643   
296       0.0336   0.007068             100             92   0.022230   
252       0.0324   0.010504             100            108   0.025570   
34        0.0314   0.014451             100             99  -2.257608   
266       0.0308   0.017417             100            144   0.030595   

         train_q99  train_min  train_max  test_min   test_max  
feature         

Now, we will recalculate Komogorov-Smirnov test, use QuantileTransformer to cap the data. Moreover, we will utilize Adversarial Validation using Random Forest to distinguish between test and train, as well as we will drop some of the features that mismatch:

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score


q_low, q_high = x_train.quantile(0.01), x_train.quantile(0.99)

stats = []
for col in x_train.columns:
    ks_stat, _ = ks_2samp(x_train[col], x_test[col])
    test_out = ((x_test[col] < q_low[col]) | (x_test[col] > q_high[col])).sum()
    stats.append({'feature': col, 'ks_stat': ks_stat, 'test_outliers': test_out})
stats_df = pd.DataFrame(stats).set_index('feature')

ks_thresh = 0.035
outlier_thresh = 120
flagged = stats_df[
    (stats_df['ks_stat'] >= ks_thresh) |
    (stats_df['test_outliers'] > outlier_thresh)
].index.tolist()
print(f"Flagged features (KS >= {ks_thresh} or test_outliers > {outlier_thresh}):\n{flagged}\n")

qt = QuantileTransformer(output_distribution='uniform', random_state=0)
combined = pd.concat([x_train[flagged], x_test[flagged]], axis=0)
qt.fit(combined)

x_train_qt = x_train.copy()
x_test_qt  = x_test.copy()
x_train_qt[flagged] = qt.transform(x_train[flagged])
x_test_qt[flagged]  = qt.transform(x_test[flagged])

X_adv = pd.concat([x_train_qt[flagged], x_test_qt[flagged]], axis=0)
y_adv = np.concatenate([np.zeros(len(x_train_qt)), np.ones(len(x_test_qt))])

X_tr, X_val, y_tr, y_val = train_test_split(
    X_adv, y_adv, test_size=0.3, random_state=0, stratify=y_adv
)
rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(X_tr, y_tr)

y_prob = rf.predict_proba(X_val)[:,1]
print(f"Adversarial AUC: {roc_auc_score(y_val, y_prob):.3f}, "
      f"Accuracy: {accuracy_score(y_val, rf.predict(X_val)):.3f}\n")

importances = pd.Series(rf.feature_importances_, index=flagged).sort_values(ascending=False)
print("Top 10 adversarial-important features:")
print(importances.head(10), "\n")

new_stats = []
for col in flagged:
    ks2, _ = ks_2samp(x_train_qt[col], x_test_qt[col])
    new_stats.append({'feature': col, 'ks_after': ks2})
new_df = pd.DataFrame(new_stats).set_index('feature')
print("KS after QuantileTransform:")
print(new_df, "\n")

to_drop = new_df[new_df['ks_after'] > ks_thresh].index.tolist()
keep_after = [f for f in flagged if f not in to_drop]
print(f"Features to drop (ks_after > {ks_thresh}): {to_drop}")
print(f"Features to keep: {keep_after}\n")

x_train_processed = x_train_qt.drop(columns=to_drop)
x_test_processed  = x_test_qt.drop(columns=to_drop)

print(f"Final shapes --> x_train: {x_train_processed.shape}, x_test: {x_test_processed.shape}")


Flagged features (KS >= 0.035 or test_outliers > 120):
[11, 20, 24, 44, 47, 50, 55, 107, 114, 117, 121, 122, 123, 127, 130, 133, 135, 163, 175, 177, 184, 188, 193, 198, 204, 214, 225, 232, 244, 253, 264, 266, 279, 342, 343, 354, 388, 390, 394, 399, 404, 413, 426, 428, 430, 434, 439, 464]

Adversarial AUC: 0.501, Accuracy: 0.490

Top 10 adversarial-important features:
130    0.022881
107    0.022392
428    0.022245
388    0.022131
135    0.021990
184    0.021944
117    0.021707
394    0.021654
204    0.021605
232    0.021597
dtype: float64 

KS after QuantileTransform:
         ks_after
feature          
11         0.0078
20         0.0186
24         0.0134
44         0.0206
47         0.0164
50         0.0206
55         0.0242
107        0.0368
114        0.0084
117        0.0218
121        0.0184
122        0.0178
123        0.0180
127        0.0196
130        0.0368
133        0.0172
135        0.0354
163        0.0120
175        0.0176
177        0.0106
184        0.0180
188        

This time, let us check Variance Threshold that are near-zero. Furthermore, we will remove highly correlated features if there exist such. We will also scale the remaining features using both train and test data:

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.feature_selection import VarianceThreshold


sel = VarianceThreshold(threshold=1e-5)
sel.fit(x_train_processed)
keep_var = x_train_processed.columns[sel.get_support()]
drop_var = [c for c in x_train_processed.columns if c not in keep_var]
print("Dropped for near-zero variance:", drop_var)
x_train_var = x_train_processed[keep_var].copy()
x_test_var  = x_test_processed[keep_var].copy()

corr = x_train_var.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
drop_corr = [col for col in upper.columns if any(upper[col] > 0.95)]
print("Dropped for high correlation:", drop_corr)
x_train_corr = x_train_var.drop(columns=drop_corr).copy()
x_test_corr  = x_test_var.drop(columns=drop_corr).copy()

scaler = StandardScaler()
combined = pd.concat([x_train_corr, x_test_corr], axis=0)
scaler.fit(combined)

x_train_final = pd.DataFrame(
    scaler.transform(x_train_corr),
    columns=x_train_corr.columns,
    index=x_train_corr.index
)
x_test_final = pd.DataFrame(
    scaler.transform(x_test_corr),
    columns=x_test_corr.columns,
    index=x_test_corr.index
)

print("Final shapes -->", x_train_final.shape, x_test_final.shape)


Dropped for near-zero variance: []
Dropped for high correlation: [7]
Final shapes --> (5000, 495) (5000, 495)


We will install hyperparameter optimization framework Optuna:

In [6]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.5 MB/s eta 0:00:00


Now, for all classifiers of choice, we will create our net_score that we want to minimize, do an optuna search to find best parameters of C and threshold. After that, we will use SHAP to rank features by importance. What is more, we will utilize the Bootstrap stability check:

In [ ]:

import numpy as np
import pandas as pd
import optuna
import shap

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train_final), columns=x_train_final.columns)
x_test_scaled = pd.DataFrame(scaler.transform(x_test_final), columns=x_test_final.columns)


def net_score(y_true, y_pred, n_features):
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(y_true, y_pred)
    return 10 * acc * len(y_true) - 200 * n_features

def count_tree_features(model):
    imp = model.feature_importances_
    return np.count_nonzero(imp)


def objective_logistic(trial):
    C = trial.suggest_float('C', 0.001, 0.01, log=True)
    threshold = trial.suggest_float('threshold', 0.3, 0.7)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fold_scores = []

    for tr_idx, val_idx in skf.split(x_train_final, y_train):
        X_tr, X_val = x_train_final.iloc[tr_idx], x_train_final.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = LogisticRegression(
            penalty='l1', solver='saga', C=C,
            max_iter=5000, random_state=0)

        model.fit(X_tr, y_tr)

        probs = model.predict_proba(X_val)[:, 1]
        preds = (probs >= threshold).astype(int)
        n_feats = np.count_nonzero(model.coef_)

        fold_scores.append(net_score(y_val, preds, n_feats))

    return np.mean(fold_scores)

print("\n--- Tuning logistic regression (L1) ---")
study_log = optuna.create_study(direction='maximize')
study_log.optimize(objective_logistic, n_trials=20)

best_log = study_log.best_params
best_C_log = best_log['C']
best_thr_log = best_log['threshold']
print("Logistic L1 best parameters:", best_log)

print("Fitting final logistic L1 model...")
model_log = LogisticRegression(
    penalty='l1', solver='saga', C=best_C_log,
    max_iter=5000, random_state=0)
model_log.fit(x_train_final, y_train)

feat_log = list(np.where(model_log.coef_[0] != 0)[0])


def objective_svm(trial):
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    threshold = trial.suggest_float('threshold', 0.3, 0.7)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fold_scores = []

    for tr_idx, val_idx in skf.split(x_train_scaled, y_train):
        X_tr, X_val = x_train_scaled.iloc[tr_idx, feat_log], x_train_scaled.iloc[val_idx, feat_log]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = SVC(C=C, kernel='linear', probability=True, random_state=0)
        model.fit(X_tr, y_tr)

        probs = model.predict_proba(X_val)[:, 1]
        preds = (probs >= threshold).astype(int)
        n_feats = len(feat_log)

        fold_scores.append(net_score(y_val, preds, n_feats))
    return np.mean(fold_scores)

print("\n--- Tuning SVM ---")
study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=15)

best_svm = study_svm.best_params
best_C_svm = best_svm['C']
best_thr_svm = best_svm['threshold']
print("SVM best parameters:", best_svm)

print("Fitting final SVM model...")
model_svm = SVC(C=best_C_svm, kernel='linear', probability=True, random_state=0)
model_svm.fit(x_train_scaled.iloc[:, feat_log], y_train)



def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    threshold = trial.suggest_float('threshold', 0.3, 0.7)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fold_scores = []

    for tr_idx, val_idx in skf.split(x_train_scaled, y_train):
        X_tr, X_val = x_train_scaled.iloc[tr_idx, feat_log], x_train_scaled.iloc[val_idx, feat_log]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = KNeighborsClassifier(n_neighbors=n_neighbors)
        model.fit(X_tr, y_tr)

        probs = model.predict_proba(X_val)[:, 1]
        preds = (probs >= threshold).astype(int)
        n_feats = len(feat_log)

        fold_scores.append(net_score(y_val, preds, n_feats))
    return np.mean(fold_scores)

print("\n--- Tuning KNN ---")
study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=15)

best_knn = study_knn.best_params
best_n_neighbors = best_knn['n_neighbors']
best_thr_knn = best_knn['threshold']
print("KNN best parameters:", best_knn)

print("Fitting final KNN model...")
model_knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)
model_knn.fit(x_train_scaled.iloc[:, feat_log], y_train)


def objective_elastic(trial):
    C = trial.suggest_float('C', 0.001, 0.1, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.1, 1.0)
    threshold = trial.suggest_float('threshold', 0.3, 0.7)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fold_scores = []

    for tr_idx, val_idx in skf.split(x_train_final, y_train):
        X_tr, X_val = x_train_final.iloc[tr_idx], x_train_final.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = LogisticRegression(
            penalty='elasticnet', solver='saga', C=C,
            l1_ratio=l1_ratio, max_iter=5000, random_state=0)
        model.fit(X_tr, y_tr)

        probs = model.predict_proba(X_val)[:, 1]
        preds = (probs >= threshold).astype(int)
        n_feats = np.count_nonzero(model.coef_)

        fold_scores.append(net_score(y_val, preds, n_feats))

    return np.mean(fold_scores)

print("\n--- Tuning logistic regression (ElasticNet) ---")
study_enet = optuna.create_study(direction='maximize')
study_enet.optimize(objective_elastic, n_trials=20)

best_enet = study_enet.best_params
best_C_enet = best_enet['C']
best_l1_ratio = best_enet['l1_ratio']
best_thr_enet = best_enet['threshold']
print("ElasticNet best parameters:", best_enet)

print("Fitting final ElasticNet model...")
model_enet = LogisticRegression(
    penalty='elasticnet', solver='saga',
    C=best_C_enet, l1_ratio=best_l1_ratio,
    max_iter=5000, random_state=0)
model_enet.fit(x_train_final, y_train)


def objective_rf(trial):
    estimators = trial.suggest_int('n_estimators', 100, 500)
    threshold = trial.suggest_float('threshold', 0.3, 0.7)
    max_features = trial.suggest_float('max_features', 0.05, 0.5)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fold_scores = []

    for tr_idx, val_idx in skf.split(x_train_final, y_train):
        X_tr, X_val = x_train_final.iloc[tr_idx], x_train_final.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = RandomForestClassifier()
        model.fit(X_tr, y_tr)

        probs = model.predict_proba(X_val)[:, 1]
        preds = (probs >= threshold).astype(int)
        n_feats = count_tree_features(model)

        fold_scores.append(net_score(y_val, preds, n_feats))

    return np.mean(fold_scores)

print("\n--- Tuning random forest ---")
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=20)

best_rf = study_rf.best_params
best_mf_rf = best_rf['max_features']
best_thr_rf = best_rf['threshold']
best_estimators_rf = best_rf['n_estimators']
print("RandomForest best parameters:", best_rf)

print("Fitting final random forest model...")
model_rf = RandomForestClassifier(
    n_estimators=best_estimators_rf,
    max_features=best_mf_rf,
    random_state=0, n_jobs=-1)
model_rf.fit(x_train_final, y_train)


def objective_xgb(trial):
    lambda_l1 = trial.suggest_float('lambda_l1', 50.0, 200.0, log=True)
    max_depth = trial.suggest_int('max_depth', 2, 4)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 0.3)
    threshold = trial.suggest_float('threshold', 0.3, 0.7)
    estimators = trial.suggest_int('n_estimators', 100, 500)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fold_scores = []

    for tr_idx, val_idx in skf.split(x_train_final, y_train):
        X_tr, X_val = x_train_final.iloc[tr_idx], x_train_final.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = XGBClassifier(
            n_estimators=estimators,
            eval_metric='logloss',
            reg_alpha=lambda_l1,
            reg_lambda=0.0,
            max_depth=max_depth,
            colsample_bytree=colsample_bytree,
            random_state=0, n_jobs=-1)
        model.fit(X_tr, y_tr)

        probs = model.predict_proba(X_val)[:, 1]
        preds = (probs >= threshold).astype(int)
        n_feats = count_tree_features(model)
        fold_scores.append(net_score(y_val, preds, n_feats))

    return np.mean(fold_scores)

print("\n--- Tuning XGBoost ---")
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=20)

best_xgb = study_xgb.best_params
best_l1_xgb = best_xgb['lambda_l1']
best_depth_xgb = best_xgb['max_depth']
best_col_xgb = best_xgb['colsample_bytree']
best_thr_xgb = best_xgb['threshold']
best_estimators_xgb = best_xgb['n_estimators']
print("XGBoost best parameters:", best_xgb)

print("Fitting final XGBoost model...")
model_xgb = XGBClassifier(
    n_estimators=best_estimators_xgb,
    eval_metric='logloss',
    reg_alpha=best_l1_xgb,
    reg_lambda=0.0,
    max_depth=best_depth_xgb,
    colsample_bytree=best_col_xgb,
    random_state=0, n_jobs=-1)
model_xgb.fit(x_train_final, y_train)


def objective_lgb(trial):
    lambda_l1 = trial.suggest_float('lambda_l1', 1.0, 100.0, log=True)
    max_depth = trial.suggest_int('max_depth', 2, 4)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 0.3)
    threshold = trial.suggest_float('threshold', 0.3, 0.7)
    estimators = trial.suggest_int('n_estimators', 100, 500)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fold_scores = []

    for tr_idx, val_idx in skf.split(x_train_final, y_train):
        X_tr, X_val = x_train_final.iloc[tr_idx], x_train_final.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = LGBMClassifier(
            n_estimators=estimators,
            reg_lambda=lambda_l1,
            reg_alpha=0.0,
            max_depth=max_depth,
            colsample_bytree=colsample_bytree,
            random_state=0, n_jobs=-1)
        model.fit(X_tr, y_tr)

        probs = model.predict_proba(X_val)[:, 1]
        preds = (probs >= threshold).astype(int)
        n_feats = count_tree_features(model)
        fold_scores.append(net_score(y_val, preds, n_feats))

    return np.mean(fold_scores)


print("\n--- Tuning LightGBM ---")
study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=20)

best_lgb = study_lgb.best_params
best_l1_lgb = best_lgb['lambda_l1']
best_depth_lgb = best_lgb['max_depth']
best_col_lgb = best_lgb['colsample_bytree']
best_thr_lgb = best_lgb['threshold']
best_estimators_lgb = best_lgb['n_estimators']
print("LightGBM best parameters:", best_lgb)

print("Fitting final LightGBM model...")
model_lgb = LGBMClassifier(
    n_estimators=best_estimators_lgb,
    reg_lambda=best_l1_lgb,
    reg_alpha=0.0,
    max_depth=best_depth_lgb,
    colsample_bytree=best_col_lgb,
    random_state=0, n_jobs=-1)
model_lgb.fit(x_train_final, y_train)


print("\nFinal model tuning results:")
print("Logistic L1: C =", best_C_log, ", threshold =", best_thr_log,
      ", # features =", np.count_nonzero(model_log.coef_))
print("SVM: C =", best_C_svm, ", threshold =", best_thr_svm,
      ", # features =", len(feat_log))
print("KNN: n_neighbors =", best_n_neighbors, ", threshold =", best_thr_knn,
      ", # features =", len(feat_log))
print("ElasticNet: C =", best_C_enet, ", l1_ratio =", best_l1_ratio,
      ", thresh =", best_thr_enet, ", #feat =", np.count_nonzero(model_enet.coef_))
print("RandomForest: max_features =", best_mf_rf,
      ", thresh =", best_thr_rf, ", # features =", count_tree_features(model_rf))
print("XGBoost: lambda_l1 =", best_l1_xgb, ", max_depth =", best_depth_xgb,
      ", colsample_bytree =", best_col_xgb,
      ", threshold =", best_thr_xgb, ", #feat =", count_tree_features(model_xgb))
print("LightGBM: lambda_l1 =", best_l1_lgb, ", max_depth =", best_depth_lgb,
      ", colsample_bytree =", best_col_lgb,
      ", threshold =", best_thr_lgb, ", # features =", count_tree_features(model_lgb))


models_info = [
    ("Logistic L1", model_log, best_thr_log, "linear"),
    ("SVM", model_svm, best_thr_svm, "linear"),
    ("KNN", model_knn, best_thr_knn, "linear"),
    ("ElasticNet", model_enet, best_thr_enet, "linear"),
    ("RandomForest", model_rf, best_thr_rf, "tree"),
    ("XGBoost", model_xgb, best_thr_xgb, "tree"),
    ("LightGBM", model_lgb, best_thr_lgb, "tree")]

for name, model, thr, model_type in models_info:
    print(f"\n--- {name} Analysis ---")
    test_probs = model.predict_proba(x_test_final)[:, 1]
    y_test_pred = (test_probs >= thr).astype(int)
    frac_pos = np.mean(y_test_pred)
    print(f"Threshold = {thr:.2f}, # positives = {int(frac_pos * len(test_probs))} / {len(test_probs)}")

    if model_type == "linear":
        explainer = shap.LinearExplainer(model, x_train_final, feature_perturbation="interventional")
        shap_vals = explainer.shap_values(x_train_final)

    else:
        explainer = shap.TreeExplainer(model, x_train_final)
        shap_vals = explainer.shap_values(x_train_final)

        if isinstance(shap_vals, list):
            shap_vals = shap_vals[1]

    mean_abs_shap = np.abs(shap_vals).mean(axis=0)
    shap_imp = pd.Series(mean_abs_shap, index=x_train_final.columns).sort_values(ascending=False)
    print("Top 10 features by mean |SHAP|:")
    print(shap_imp.head(10))

    B = 100
    rng = np.random.RandomState(0)
    feat_count = pd.Series(0, index=x_train_final.columns)

    for i in range(B):
        idx = rng.choice(len(x_train_final), len(x_train_final), replace=True)
        Xb, yb = x_train_final.iloc[idx], y_train.iloc[idx]

        if model_type == "linear":
            m = LogisticRegression(
                penalty='l1' if name=="Logistic L1" else 'elasticnet',
                solver='saga',
                C=(best_C_log if name=="Logistic L1" else best_C_enet),
                l1_ratio=(best_l1_ratio if name=="ElasticNet" else None),
                max_iter=5000, random_state=i)

        elif name == "RandomForest":
            m = RandomForestClassifier(
                n_estimators=100, max_features=best_mf_rf,
                random_state=i, n_jobs=-1)

        elif name == "XGBoost":
            m = XGBClassifier(
                n_estimators=100,
                eval_metric='logloss',
                reg_lambda=best_l1_xgb, max_depth=best_depth_xgb,
                colsample_bytree=best_col_xgb,
                random_state=i, n_jobs=-1)

        else:
            m = LGBMClassifier(
                n_estimators=100,
                reg_lambda=best_l1_lgb,
                max_depth=best_l1_lgb,
                colsample_bytree=best_col_lgb,
                random_state=i, n_jobs=-1)

        m.fit(Xb, yb)

        if model_type == "linear":
            sel = x_train_final.columns[m.coef_[0] != 0]

        else:
            sel = x_train_final.columns[m.feature_importances_ > 0]
        feat_count[sel] += 1

    stability = (feat_count / B).sort_values(ascending=False)
    print("Bootstrap selection frequency (top 10):")
    print(stability.head(10))


feat_nb = list(range(x_train_final.shape[1]))
feat_enet = list(np.where(model_enet.coef_[0] != 0)[0])
feat_rf = list(np.where(model_rf.feature_importances_ > 0)[0])
feat_xgb = list(np.where(model_xgb.feature_importances_ > 0)[0])
feat_lgb = list(np.where(model_lgb.feature_importances_ > 0)[0])


model_dicts = [
    {'name': 'Logistic L1', 'model': LogisticRegression, 'params': {'penalty': 'l1', 'solver': 'saga', 'C': best_C_log, 'max_iter': 10000, 'random_state': 0}},
    {'name': 'SVM', 'model': SVC, 'params': {'C': best_C_svm, 'kernel': 'linear', 'probability': True, 'random_state': 0}},
    {'name': 'KNN', 'model': KNeighborsClassifier, 'params': {'n_neighbors': best_n_neighbors}},
    {'name': 'ElasticNet', 'model': LogisticRegression, 'params': {'penalty': 'elasticnet', 'solver': 'saga', 'C': best_C_enet, 'l1_ratio': best_l1_ratio, 'max_iter': 10000, 'random_state': 0}},
    {'name': 'RandomForest', 'model': RandomForestClassifier, 'params': {'n_estimators': best_estimators_rf, 'max_features': best_mf_rf, 'random_state': 0, 'n_jobs': -1}},
    {'name': 'XGBoost', 'model': XGBClassifier, 'params': {'n_estimators': best_estimators_xgb, 'reg_alpha': best_l1_xgb, 'reg_lambda': 0.0, 'max_depth': best_depth_xgb, 'colsample_bytree': best_col_xgb, 'eval_metric': 'logloss', 'random_state': 0, 'n_jobs': -1}},
    {'name': 'LightGBM', 'model': LGBMClassifier, 'params': {'n_estimators': best_estimators_lgb, 'reg_lambda': best_l1_lgb, 'reg_alpha': 0.0, 'max_depth': best_depth_lgb, 'colsample_bytree': best_col_lgb, 'random_state': 0, 'n_jobs': -1}},]

feature_dicts = {
    'Logistic L1': feat_log,
    'SVM': feat_log,
    'KNN': feat_log,
    'ElasticNet': feat_enet,
    'RandomForest': feat_rf,
    'XGBoost': feat_xgb,
    'LightGBM': feat_lgb,}

[I 2025-06-02 12:17:43,249] A new study created in memory with name: no-name-fde4bbf8-8c84-4780-8ca0-56e5748cc90c



--- Tuning logistic regression (L1) ---


[I 2025-06-02 12:17:46,152] Trial 0 finished with value: 5202.0 and parameters: {'C': 0.001854123988286725, 'threshold': 0.5754726744893557}. Best is trial 0 with value: 5202.0.
[I 2025-06-02 12:17:48,975] Trial 1 finished with value: 5420.0 and parameters: {'C': 0.0029568484555345595, 'threshold': 0.3766128268257427}. Best is trial 1 with value: 5420.0.
[I 2025-06-02 12:17:51,934] Trial 2 finished with value: 6862.0 and parameters: {'C': 0.0025830842130550517, 'threshold': 0.4786808089002411}. Best is trial 2 with value: 6862.0.
[I 2025-06-02 12:17:56,080] Trial 3 finished with value: 5470.0 and parameters: {'C': 0.006107622334077325, 'threshold': 0.3390098029677393}. Best is trial 2 with value: 6862.0.
[I 2025-06-02 12:17:59,467] Trial 4 finished with value: 6766.0 and parameters: {'C': 0.005081173469315885, 'threshold': 0.4978502177671862}. Best is trial 2 with value: 6862.0.
[I 2025-06-02 12:18:05,172] Trial 5 finished with value: 5352.0 and parameters: {'C': 0.009127768061167345, 

Logistic L1 best parameters: {'C': 0.0025830842130550517, 'threshold': 0.4786808089002411}
Fitting final logistic L1 model...


[I 2025-06-02 12:18:51,391] A new study created in memory with name: no-name-86e5ec3e-1552-4607-8182-a9ee62bf69f3


...

[I 2025-06-02 12:22:13,360] Trial 0 finished with value: -67830.0 and parameters: {'C': 0.09675828747147912, 'l1_ratio': 0.7937960359603429, 'threshold': 0.6758000155546047}. Best is trial 0 with value: -67830.0.
[I 2025-06-02 12:22:16,368] Trial 1 finished with value: 4914.0 and parameters: {'C': 0.0010007096409628984, 'l1_ratio': 0.6860914744718605, 'threshold': 0.6454157477511124}. Best is trial 1 with value: 4914.0.
[I 2025-06-02 12:22:34,046] Trial 2 finished with value: -61894.0 and parameters: {'C': 0.025834433591715254, 'l1_ratio': 0.2845030527179061, 'threshold': 0.6553203438225808}. Best is trial 1 with value: 4914.0.
[I 2025-06-02 12:22:40,238] Trial 3 finished with value: -31808.0 and parameters: {'C': 0.006158281263123593, 'l1_ratio': 0.15974649441278216, 'threshold': 0.6212389189420671}. Best is trial 1 with value: 4914.0.
[I 2025-06-02 12:23:44,587] Trial 4 finished with value: -75194.0 and parameters: {'C': 0.09621264523036395, 'l1_ratio': 0.5593722349571608, 'threshold

ElasticNet best parameters: {'C': 0.006710491299252708, 'l1_ratio': 0.980956554686088, 'threshold': 0.47609595295077023}
Fitting final ElasticNet model...


[I 2025-06-02 12:26:10,533] A new study created in memory with name: no-name-76b07fd5-df86-4b24-94df-f536133abf8a



--- Tuning random forest ---


[I 2025-06-02 12:27:10,390] Trial 0 finished with value: -92350.0 and parameters: {'n_estimators': 383, 'threshold': 0.38438911230672346, 'max_features': 0.33334267205391954}. Best is trial 0 with value: -92350.0.
[I 2025-06-02 12:28:10,664] Trial 1 finished with value: -92098.0 and parameters: {'n_estimators': 382, 'threshold': 0.5661748181956261, 'max_features': 0.46947865745297435}. Best is trial 1 with value: -92098.0.
[I 2025-06-02 12:29:10,669] Trial 2 finished with value: -93220.0 and parameters: {'n_estimators': 391, 'threshold': 0.3008161125968057, 'max_features': 0.486591286945903}. Best is trial 1 with value: -92098.0.
[I 2025-06-02 12:30:11,620] Trial 3 finished with value: -91974.0 and parameters: {'n_estimators': 251, 'threshold': 0.5177025277810962, 'max_features': 0.16683356880330458}. Best is trial 3 with value: -91974.0.
[I 2025-06-02 12:31:12,350] Trial 4 finished with value: -92698.0 and parameters: {'n_estimators': 425, 'threshold': 0.6448104633241611, 'max_feature

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def net_score(y_true, y_pred, n_features):
    acc = accuracy_score(y_true, y_pred)
    return 10 * acc * len(y_true) - 200 * n_features

def evaluate_model_on_features(model_class, model_params, feats, X, y):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    all_true, all_prob = [], []
    for tr_idx, val_idx in skf.split(X, y):
        X_tr = X.iloc[tr_idx, feats]
        y_tr = y.iloc[tr_idx]
        X_val = X.iloc[val_idx, feats]
        y_val = y.iloc[val_idx]
        model = model_class(**model_params)
        model.fit(X_tr, y_tr)
        all_true.append(y_val.values)
        all_prob.append(model.predict_proba(X_val)[:, 1])
    all_true = np.concatenate(all_true)
    all_prob = np.concatenate(all_prob)
    best_net, best_thr = -np.inf, 0.5
    for thr in np.linspace(0.1, 0.9, 81):
        preds = (all_prob >= thr).astype(int)
        net = net_score(all_true, preds, len(feats))
        if net > best_net:
            best_net, best_thr = net, thr
    return best_net, best_thr

results = []

for model_info in model_dicts:
    name = model_info['name']
    feats = feature_dicts[name]
    if len(feats) == 0:
        print(f"{name}: No selected features, skipping.")
        continue
    net, thr = evaluate_model_on_features(model_info['model'], model_info['params'], feats, x_train_final, y_train)
    print(f"{name}: net-score = {net:.1f} @ threshold {thr:.2f} with {len(feats)} features.")
    results.append({'name': name, 'feats': feats, 'net_score': net, 'threshold': thr})

best_result = max(results, key=lambda d: d['net_score'])
print(f"\nBest model: {best_result['name']} with net-score {best_result['net_score']:.1f}, {len(best_result['feats'])} features, and threshold {best_result['threshold']:.2f}")

# final model trained on ALL data with best subset and threshold
final_model_info = next(item for item in model_dicts if item['name'] == best_result['name'])
final_model = final_model_info['model'](**final_model_info['params'])
final_model.fit(x_train_final.iloc[:, best_result['feats']], y_train)

# predict on test set
test_probs = final_model.predict_proba(x_test_final.iloc[:, best_result['feats']])[:, 1]

top_1000_idx = np.argsort(-test_probs)[:1000]

student_id = '323597'
np.savetxt(f"{student_id}_obs.txt", top_1000_idx, fmt='%d')
np.savetxt(f"{student_id}_vars.txt", best_result['feats'], fmt='%d')
